In [1]:
import pormake as pm
import numpy as np
import csv
from collections import defaultdict

db = pm.Database()
loc = pm.Locator()

pm.log.disable_print()
pm.log.disable_file_print()


2023-07-24 09:53:27.245314: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 09:53:27.279446: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-24 09:53:27.280366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 09:53:27.957731: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
>>> Console logs (under WARNING level) are disabled.


In [2]:
def expand_topo(topo):
    x = 1
    y = 1
    z = 1
    edges = topo.edge_indices
    neigh = topo.neighbor_list
    for i in edges:

        ## below set lists unique neighbors for the edges. If 1, this means expansion is required.
        if len(set([j.index for j in neigh[i]])) == 1:
            dist = [abs(d) for d in neigh[i][0].distance_vector]

            ## expand topology along the "major" edge direction to allow for alternating node sequence.
            if x == 1 and dist.index(max(dist)) == 0:
                x = 2
            if y == 1 and dist.index(max(dist)) == 1:
                y = 2
            if z == 1 and dist.index(max(dist)) == 2:
                z = 2

    return topo * (x, y, z)

In [3]:
## prep topologies for patterned node assignment

topo_codes = []
file = open('cn34_pid_topology_list.csv')
csvreader = csv.reader(file)

for row in csvreader:
    for topo in row:
        topo_codes.append(topo)

topos = []

for topo in topo_codes:
    cur_topo = db.get_topo(topo)
    cur_topo = expand_topo(cur_topo)

    if cur_topo.n_nodes % 2 == 1:
        ## expand in one of the directions to ensure even number of nodes
        cur_topo = cur_topo * (1, 1, 2)
        
    topos.append(cur_topo)

In [7]:
## check rmsd and topology scale here (check # nodes again since expansion took place)

node_limit = 80
refined_topos = []

for topo in topos:
    
    if topo.n_nodes > node_limit:
        continue
    refined_topos.append(topo)


In [10]:
## attempt generation!
test_builder = pm.Builder(planarity_enforcement=True, angle_threshold=10, check_tetrahedral=True)

TP_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/TP_O.xyz")
PcM_O = pm.BuildingBlock(bb_file="pi-d_building_blocks/PcM_O.xyz")
Fe_pln = pm.BuildingBlock(bb_file="pi-d_building_blocks/Fe_pln.xyz")

successful_cases = []

for topo in refined_topos:
    
    print(topo.name)
        
    current_node = {}
    current_edge = {}    
    
    ## initialize nodes with Fe_oct first (no chirality dependence... is this good?)
    for i, node in enumerate(topo.unique_cn):
        
        if node == 3:
            current_node[i] = TP_O
        elif node == 4:
            current_node[i] = PcM_O
        else:
            print('something wrong with', topo.name)

    
    ## initialize edges
    for i, edge in enumerate(topo.unique_edge_types):
        current_edge[tuple(edge)] = Fe_pln

    bbs = test_builder.make_bbs_by_type(topo, current_node, current_edge)
    calc_permutation = test_builder.extract_permutation(topo, current_node, current_edge)
    calc_permutation = {i : perm for i, perm in enumerate(calc_permutation)}
    
    try:
        mof = test_builder.build(topo, bbs, calc_permutation)
        mof.view()
        successful_cases.append(topo.name)

    except:
        continue


>>> There are atoms without bond: TP_O, [(33, 'Xx'), (34, 'Xx'), (35, 'Xx')].
>>> There are atoms without bond: PcM_O, [(64, 'Xx'), (56, 'Ni'), (61, 'Xx'), (62, 'Xx'), (63, 'Xx')].
>>> There are atoms without bond: Fe_pln, [(3, 'Xx'), (4, 'Xx')].


bcu-f
bcu-h
bpa
bpb
bpc
bpe
bpg
bpi
bpl
bpm
bpp
bps
bpt
bpv
bto
cdl
cdm
cdn
cds
cds-b
cdz
clh
ctn-d
eta
eta-c
etb
etc
etd
ete
etf
etg
eth
etj
etn
ffc
fjh
fof
fog
gwg
hha
hof
iab
jph
lig
lil
lwg
mot
mou
nbo
nnd
nod
noh
noi
noj
nos
nta
ntb
nva
nvb
ofo
oft
pbp
pbz
pcu-g
pcu-h
phl
pto
pyo
qdl
qzd
srd
srd-l
srs
ssd
sse
ssf
sst
stu
stx
sur
sxc-d
tfc
tfe
tfn
tfo
tfp
tfz
ths
tta
ttg
twt
uks
uni-d
uom
utg
utj
utm
utp
wix
wiz
zme
zyg


In [11]:
successful_cases

['bpc',
 'bpi',
 'cdl',
 'cdm',
 'cdn',
 'cds',
 'cds-b',
 'cdz',
 'clh',
 'etc',
 'hof',
 'mot',
 'mou',
 'nbo',
 'nta',
 'ssd',
 'sse',
 'ssf',
 'sst',
 'tfc',
 'tfe',
 'tfn',
 'tfz',
 'ths',
 'twt',
 'utp']